In [86]:
# we need the following libraries, so let's install them
%pip install boto3
%pip install torchxrayvision
%pip install SimpleITK
%pip install radiomics
%pip install pyradiomics
%pip install torchvision
%pip install matplotlib
%pip install scikit-learn
%pip install tensorflow


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached pyradiomics-3.1.0.tar.gz (34.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://f

In [96]:
# importing libraries
from time import time
import multiprocessing
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from glob import glob
import skimage
import matplotlib.pyplot as plt
import sys
import warnings
from sklearn.utils import shuffle
import random
import logging
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore", category=DeprecationWarning)
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)

In [97]:
# here we call the directories that images are saved; features are saved; targets are saved.
directory_ = '/content/sample_data/CheXpert/'
directory_processed_images = directory_ + 'processed_images'
directory_filterss = directory_ + 'filters'
directory_segments = directory_ + 'segments'
directory_cleaned_targets = directory_ + 'cleaned_targets'
directory_features = directory_ + 'features'

In [103]:
# loadnig the data and removing the useless column 'Unnamed: 0'
whole_data = pd.read_csv('synced_data.csv')
del whole_data['Unnamed: 0']
whole_data.head()
whole_data = whole_data.sample(frac=1.0, random_state=21202)

In [104]:
# finding total number of samples:
n_samples = whole_data.shape[0]

In [105]:
# names of all targets:
targets_names = list(whole_data.iloc[:,464:])
print(targets_names)

['index', 'Path', 'Patient_id', 'Study_Number', 'Sex', 'Age', 'Race', 'Ethnicity', 'Frontal/Lateral', 'AP/PA', 'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']


In [106]:
# finding the number of unique subjects:
unique_subjects = pd.unique(whole_data['Patient_id'])
print('The number of unique subjects are: {0}'.format(len(unique_subjects)))

The number of unique subjects are: 500


In [107]:
# preallocating the frequency of session of the subjects:
# this is written to make sure that we dont use a subject in both training and testing split.
test_split = 0.06       # precent out of total samples
valid_split = 0.1       # precent out of total samples (this is the sumation of both training and validation splits)

n_test = n_samples * test_split
n_whole_train = n_samples - n_test

frequencies = np.ones((unique_subjects.shape[0], 4)) * np.nan
for i, sbj in tqdm(enumerate(unique_subjects)):
  frequencies[i, 0] = sbj
  frequencies[i, 1] = (whole_data['Patient_id'] == sbj).sum()
cumolative_sums_of_samples = np.cumsum(frequencies[:,1])
frequencies[:, 2] = cumolative_sums_of_samples
frequencies[:, 3] = cumolative_sums_of_samples/cumolative_sums_of_samples[~0]

500it [00:00, 6285.84it/s]


In [108]:
# visualizing the frequency  of subjects and images taken from each
vis_frequencies = pd.DataFrame(frequencies,columns=['subject_ID','n_of_sessions/images','cumolative_sessions','preccent_of_number_of_images'])
vis_frequencies

,subject_ID,n_of_sessions/images,cumolative_sessions,preccent_of_number_of_images
0,419.0,41.0,41.0,0.002733
1,219.0,35.0,76.0,0.005067
2,213.0,39.0,115.0,0.007667
3,101.0,29.0,144.0,0.009600
4,411.0,28.0,172.0,0.011467
...,...,...,...,...
495,16.0,23.0,14901.0,0.993400
496,32.0,22.0,14923.0,0.994867
497,321.0,25.0,14948.0,0.996533
498,85.0,28.0,14976.0,0.998400


In [109]:
# separating the testing split without having any mutual_subjects:
testing_subjects_masking = vis_frequencies['preccent_of_number_of_images'] < test_split
testing_subjects = vis_frequencies[testing_subjects_masking]['subject_ID'].values.tolist()
whole_train_subjects = vis_frequencies[~testing_subjects_masking]['subject_ID'].values.tolist()

In [110]:
# separating test_data and whole_trainig_data
Test = whole_data[whole_data['Patient_id'].isin(testing_subjects)]
X_test, y_test = Test.iloc[:,:464].values, Test['Age'].values

# separating test_data and whole_trainig_data
Train = whole_data[whole_data['Patient_id'].isin(whole_train_subjects)]

# the size of training/testing_split:
print('The size of the trainig split is: {}'.format(Test.shape))
print('The size of the trainig split is: {}'.format(Train.shape))

The size of the trainig split is: (893, 488)
The size of the trainig split is: (14107, 488)


In [111]:
# the size of testing_split:
print('The size of the testing split is: {}'.format(Test.shape))

The size of the testing split is: (893, 488)


In [112]:
# splitting the validation split:
X_train, X_val, y_train, y_val = train_test_split(Train.iloc[:,:464].values, Train['Age'].values, test_size=valid_split)

In [113]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((12696, 464), (1411, 464), (12696,), (1411,))

In [114]:
# normalizing the data
mean_val = X_train.mean(axis=0)
std_val = X_train.std(axis=0)

X_train -= mean_val
X_train /= std_val

X_val -= mean_val
X_val /= std_val

X_test -= mean_val
X_test /= std_val

In [119]:
# defining the model structure for the age
# %% building uncompiled mdodel

checkpoint_filepath = os.getcwd()  + '/mdl.h5'
model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=1
                                                            )


def classifer_1(features_number: int) -> callable:
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=40, activation="relu", input_dim=features_number))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(units=40, activation="relu"))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(units=1))
    model.summary()
    return model

def compiler_1(X_train_, Y_train_, X_valid_, Y_valid_):
  initial_learning_rate = 0.00001
  epochs = 300
  batch_size = 64
  model = classifer_1(features_number=464)
  # lss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
  # metrics = ['accuracy',
  #                       tf.keras.metrics.AUC(curve='PR',name='PR-curve'),
  #                       tf.keras.metrics.AUC(curve='ROC',name='ROC-curve')]
  metrics = metrics=[tf.keras.metrics.RootMeanSquaredError()]
  lss = 'mean_absolute_error'
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=initial_learning_rate,
                                                beta_1=0.9,
                                                beta_2=0.999,
                                                epsilon=1e-08),
                                                loss=lss,
                metrics=metrics)
  model.fit(x=X_train_,
            y=Y_train_,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_valid_, Y_valid_),
            verbose=2,
            shuffle=True,
            callbacks=[model_checkpoint_callback_1])
  return model

In [120]:
model_1 = compiler_1(X_train_ = X_train,
                    Y_train_ = y_train,
                    X_valid_ = X_val,
                    Y_valid_ = y_val)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 40)                18600     
                                                                 
 dropout_2 (Dropout)         (None, 40)                0         
                                                                 
 dense_4 (Dense)             (None, 40)                1640      
                                                                 
 dropout_3 (Dropout)         (None, 40)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 41        
                                                                 
Total params: 20281 (79.22 KB)
Trainable params: 20281 (79.22 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/300

Epoch 1: val_loss improved from inf to 31.58504, saving model to /Users/mra/Desktop/for_hadis/mdl.h5
199/199 - 1s - loss: 31.7766 - root_mean_squared_error: 34.3504 - val_loss: 31.5850 - val_root_mean_squared_error: 34.1628 - 957ms/epoch - 5ms/step
Epoch 2/300

Epoch 2: val_loss improved from 31.58504 to 31.37851, saving model to /Users/mra/Desktop/for_hadis/mdl.h5
199/199 - 0s - loss: 31.5854 - root_mean_squared_error: 34.1694 - val_loss: 31.3785 - val_root_mean_squared_error: 33.9731 - 453ms/epoch - 2ms/step
Epoch 3/300

Epoch 3: val_loss improved from 31.37851 to 31.16794, saving model to /Users/mra/Desktop/for_hadis/mdl.h5
199/199 - 0s - loss: 31.3796 - root_mean_squared_error: 33.9837 - val_loss: 31.1679 - val_root_mean_squared_error: 33.7800 - 488ms/epoch - 2ms/step
Epoch 4/300

Epoch 4: val_loss improved from 31.16794 to 30.95292, saving model to /Users/mra/Desktop/for_hadis/mdl.h5
199/199 - 0s - loss: 31.1568 - root_mean_squared_error: 33.7829 - val_loss: 30.9529 -

KeyboardInterrupt: 